# 🎓 RIS PhD Ultimate Research Dashboard

## Comprehensive, Fully Customizable Research Platform

**Version:** 1.0.0  
**Purpose:** Professional-grade research platform for RIS probe-based ML experiments

### Features:
- ✅ 5 Customization Tabs (System, Model, Training, Evaluation, Visualization)
- ✅ 19 Pre-defined Model Architectures + Custom
- ✅ 6 Probe Types (continuous, binary, 2bit, hadamard, sobol, halton)
- ✅ 25+ Plot Types
- ✅ Multi-Model Comparison
- ✅ Multi-Seed Statistical Analysis
- ✅ Config Save/Load (JSON/YAML)

---

## Cell 1: Setup & Installation Check

Verify environment setup and dependencies.

In [ ]:
# Check if running from repo root
import os
import sys
from pathlib import Path

# Get notebook directory
notebook_dir = Path.cwd()
repo_root = notebook_dir.parent if notebook_dir.name == 'notebooks' else notebook_dir

# Add repo root to path
if str(repo_root) not in sys.path:
    sys.path.insert(0, str(repo_root))

print(f"📁 Notebook directory: {notebook_dir}")
print(f"📁 Repository root: {repo_root}")
print(f"📁 Working directory: {os.getcwd()}")

# Change to repo root if needed
if os.getcwd() != str(repo_root):
    os.chdir(repo_root)
    print(f"✅ Changed working directory to: {os.getcwd()}")

# Verify all dependencies
print("\n" + "="*70)
print("DEPENDENCY CHECK")
print("="*70)

dependencies = {
    'numpy': 'numpy',
    'torch': 'torch',
    'matplotlib': 'matplotlib',
    'seaborn': 'seaborn',
    'tqdm': 'tqdm',
    'pandas': 'pandas',
    'scipy': 'scipy',
    'ipywidgets': 'ipywidgets',
    'yaml': 'pyyaml',
}

missing = []
for module, package in dependencies.items():
    try:
        __import__(module)
        print(f"✅ {package:15s} installed")
    except ImportError:
        print(f"❌ {package:15s} NOT FOUND")
        missing.append(package)

if missing:
    print(f"\n⚠️  Missing packages: {', '.join(missing)}")
    print("\nInstall with: pip install " + " ".join(missing))
else:
    print("\n✅ All dependencies installed!")

# System info
print("\n" + "="*70)
print("SYSTEM INFORMATION")
print("="*70)

import torch
import numpy as np

print(f"Python version:  {sys.version.split()[0]}")
print(f"NumPy version:   {np.__version__}")
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available:  {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"CUDA version:    {torch.version.cuda}")
    print(f"GPU device:      {torch.cuda.get_device_name(0)}")
else:
    print("Running on CPU")

print("\n✅ Setup complete!")

## Cell 2: Import Dashboard Components

Load the dashboard system and initialize widgets.

In [ ]:
# Import dashboard components
import dashboard
from dashboard import (
    get_all_widgets,
    create_tab_layout,
    setup_all_callbacks,
    reset_to_defaults,
    get_validation_errors,
    config_to_dict,
    dict_to_widgets,
    save_config,
    load_config,
    run_single_experiment,
    run_multi_model_comparison,
    run_multi_seed_experiment,
    aggregate_results,
)

# Import plotting
from dashboard.plots import EXTENDED_PLOT_REGISTRY

# Import ipywidgets
import ipywidgets as widgets
from IPython.display import display, clear_output

# Print welcome
dashboard.print_welcome()

# Initialize widgets
print("Initializing widget system...")
widgets_dict = get_all_widgets()
setup_all_callbacks(widgets_dict)
print("✅ Widget system initialized!")
print(f"✅ {len(EXTENDED_PLOT_REGISTRY)} plot types available")

## Cells 3-7: Widget-Based Control Panel (5 Tabs)

### Comprehensive Parameter Configuration

Use the tabs below to configure all aspects of your experiment:

1. **⚙️ System & Physics** - RIS elements, codebook size, sensing budget, channel parameters, probe types
2. **🧠 Model Architecture** - Pre-defined models or custom architecture, dropout, activations
3. **📊 Training Config** - Dataset sizes, batch size, learning rate, optimizer, scheduler
4. **📈 Evaluation** - Top-m values, model comparison, multi-seed runs
5. **🎨 Visualization** - Plot selection, output format, styling

**💡 Tip:** Hover over widgets for descriptions. Changes are validated automatically.

In [ ]:
# Display tabbed interface
tabs = create_tab_layout()
display(tabs)

### Parameter Reference Table

| Category | Parameter | Default | Range | Description |
|----------|-----------|---------|-------|-------------|
| **System** | N | 32 | 4-256 | Number of RIS elements |
| | K | 64 | 4-512 | Total probes in codebook |
| | M | 8 | 1-K | Sensing budget (probes measured) |
| | P_tx | 1.0 | 0.1-10 | Transmit power |
| | probe_type | continuous | 6 options | Probe generation method |
| **Model** | hidden_sizes | [512,256,128] | varies | Layer architecture |
| | dropout_prob | 0.1 | 0-0.8 | Dropout regularization |
| **Training** | n_train | 50000 | 1000+ | Training samples |
| | learning_rate | 1e-3 | 1e-5 to 1e-1 | Learning rate |
| | batch_size | 128 | 32-512 | Batch size |
| | n_epochs | 50 | 1-500 | Maximum epochs |
| **Eval** | top_m_values | [1,2,4,8] | 1-K | Top-m accuracy metrics |

## Cell 8: Action Buttons & Status

Control experiment execution and configuration management.

In [ ]:
# Global variables for experiment state
current_results = None
current_config = None

# Define button callbacks
def on_run_experiment_clicked(b):
    """Execute experiment with current configuration."""
    global current_results, current_config
    
    # Clear previous output
    widgets_dict['status_output'].clear_output()
    widgets_dict['results_summary'].value = "<div style='padding:10px;'><i>Running experiment...</i></div>"
    widgets_dict['results_plots'].clear_output()
    
    with widgets_dict['status_output']:
        try:
            print("🚀 Starting experiment...\n")
            
            # Get configuration from widgets
            config_dict = config_to_dict(widgets_dict)
            current_config = config_dict
            
            # Validate configuration
            is_valid, errors = get_validation_errors(config_dict)
            if not is_valid:
                print("❌ Configuration validation failed:\n")
                for error in errors:
                    print(f"  - {error}")
                return
            
            print("✅ Configuration validated\n")
            
            # Progress callback
            def progress_callback(epoch, total_epochs, metrics):
                progress = int((epoch / total_epochs) * 100)
                widgets_dict['progress_bar'].value = progress
                
                metrics_html = f"""<div style='font-family: monospace; padding: 10px;'>
                <b>Epoch {epoch}/{total_epochs}</b><br>
                Train Loss: {metrics.get('train_loss', 0):.4f}<br>
                Val Loss: {metrics.get('val_loss', 0):.4f}<br>
                Val Accuracy: {metrics.get('val_acc', 0):.3f}<br>
                Val η: {metrics.get('val_eta', 0):.4f}<br>
                </div>"""
                widgets_dict['live_metrics'].value = metrics_html
            
            # Check if multi-model comparison
            if config_dict.get('compare_multiple_models', False):
                models_to_compare = list(config_dict.get('models_to_compare', []))
                if len(models_to_compare) >= 2:
                    print(f"Running multi-model comparison: {len(models_to_compare)} models\n")
                    current_results = run_multi_model_comparison(
                        base_config_dict=config_dict,
                        model_names=models_to_compare,
                        progress_callback=progress_callback,
                        verbose=True
                    )
                else:
                    print("⚠️ Need at least 2 models for comparison. Running single experiment.\n")
                    current_results = run_single_experiment(
                        config_dict=config_dict,
                        progress_callback=progress_callback,
                        verbose=True
                    )
            # Check if multi-seed runs
            elif config_dict.get('multi_seed_runs', False):
                num_seeds = config_dict.get('num_seeds', 3)
                base_seed = config_dict.get('seed', 42)
                seeds = [base_seed + i for i in range(num_seeds)]
                
                print(f"Running multi-seed experiment: {num_seeds} seeds\n")
                current_results = run_multi_seed_experiment(
                    config_dict=config_dict,
                    seeds=seeds,
                    progress_callback=progress_callback,
                    verbose=True
                )
            else:
                # Single experiment
                current_results = run_single_experiment(
                    config_dict=config_dict,
                    progress_callback=progress_callback,
                    verbose=True
                )
            
            print("\n✅ Experiment completed successfully!")
            widgets_dict['progress_bar'].value = 100
            
            # Update results display
            update_results_display()
            
        except Exception as e:
            print(f"\n❌ Experiment failed with error:\n{str(e)}")
            import traceback
            traceback.print_exc()


def on_save_config_clicked(b):
    """Save current configuration to file."""
    widgets_dict['status_output'].clear_output()
    
    with widgets_dict['status_output']:
        try:
            config_dict = config_to_dict(widgets_dict)
            
            # Create configs directory if needed
            os.makedirs('configs', exist_ok=True)
            
            # Save as both JSON and YAML
            import time
            timestamp = time.strftime("%Y%m%d_%H%M%S")
            json_path = f"configs/config_{timestamp}.json"
            yaml_path = f"configs/config_{timestamp}.yaml"
            
            if save_config(config_dict, json_path):
                print(f"✅ Configuration saved to: {json_path}")
            
            if save_config(config_dict, yaml_path):
                print(f"✅ Configuration saved to: {yaml_path}")
                
        except Exception as e:
            print(f"❌ Failed to save configuration: {str(e)}")


def on_load_config_clicked(b):
    """Load configuration from file."""
    widgets_dict['status_output'].clear_output()
    
    with widgets_dict['status_output']:
        try:
            # List available configs
            if os.path.exists('configs'):
                files = sorted([f for f in os.listdir('configs') 
                              if f.endswith(('.json', '.yaml', '.yml'))])
                
                if files:
                    print("Available configuration files:")
                    for i, f in enumerate(files, 1):
                        print(f"  {i}. {f}")
                    print("\nTo load a config, use: dict_to_widgets(load_config('configs/filename.json'), widgets_dict)")
                else:
                    print("No saved configurations found in 'configs/' directory.")
            else:
                print("No 'configs/' directory found. Save a configuration first.")
                
        except Exception as e:
            print(f"❌ Failed to load configuration: {str(e)}")


def on_reset_defaults_clicked(b):
    """Reset all widgets to default values."""
    widgets_dict['status_output'].clear_output()
    
    with widgets_dict['status_output']:
        print("Resetting all parameters to defaults...")
        reset_to_defaults(widgets_dict)
        print("\n✅ All parameters reset to defaults!")


def update_results_display():
    """Update the results display with experiment results."""
    global current_results
    
    if current_results is None:
        return
    
    # Generate summary
    if isinstance(current_results, dict):  # Multi-model comparison
        summary_html = "<div style='padding: 10px; font-family: sans-serif;'>"
        summary_html += "<h3>Multi-Model Comparison Results</h3>"
        summary_html += "<table style='border-collapse: collapse; width: 100%;'>"
        summary_html += "<tr style='background-color: #f0f0f0;'>"
        summary_html += "<th style='border: 1px solid #ddd; padding: 8px;'>Model</th>"
        summary_html += "<th style='border: 1px solid #ddd; padding: 8px;'>η_top1</th>"
        summary_html += "<th style='border: 1px solid #ddd; padding: 8px;'>Accuracy</th>"
        summary_html += "<th style='border: 1px solid #ddd; padding: 8px;'>Time (s)</th>"
        summary_html += "</tr>"
        
        for model_name, result in current_results.items():
            summary_html += "<tr>"
            summary_html += f"<td style='border: 1px solid #ddd; padding: 8px;'>{model_name}</td>"
            summary_html += f"<td style='border: 1px solid #ddd; padding: 8px;'>{result.evaluation.eta_top1:.4f}</td>"
            summary_html += f"<td style='border: 1px solid #ddd; padding: 8px;'>{result.evaluation.accuracy_top1:.3f}</td>"
            summary_html += f"<td style='border: 1px solid #ddd; padding: 8px;'>{result.execution_time:.1f}</td>"
            summary_html += "</tr>"
        
        summary_html += "</table></div>"
        
    elif isinstance(current_results, list):  # Multi-seed
        agg = aggregate_results(current_results)
        summary_html = "<div style='padding: 10px; font-family: sans-serif;'>"
        summary_html += "<h3>Multi-Seed Experiment Results</h3>"
        summary_html += f"<p><b>Number of runs:</b> {agg['n_runs']}</p>"
        summary_html += f"<p><b>η_top1:</b> {agg['eta_top1']['mean']:.4f} ± {agg['eta_top1']['std']:.4f}</p>"
        summary_html += f"<p><b>Accuracy:</b> {agg['accuracy_top1']['mean']:.3f} ± {agg['accuracy_top1']['std']:.3f}</p>"
        summary_html += "</div>"
        
    else:  # Single experiment
        result = current_results
        summary_html = "<div style='padding: 10px; font-family: sans-serif;'>"
        summary_html += "<h3>Experiment Results</h3>"
        summary_html += f"<p><b>η_top1:</b> {result.evaluation.eta_top1:.4f}</p>"
        summary_html += f"<p><b>Accuracy (Top-1):</b> {result.evaluation.accuracy_top1:.3f}</p>"
        summary_html += f"<p><b>Accuracy (Top-4):</b> {result.evaluation.accuracy_top4:.3f}</p>"
        summary_html += f"<p><b>Execution time:</b> {result.execution_time:.1f}s</p>"
        summary_html += "</div>"
    
    widgets_dict['results_summary'].value = summary_html
    
    # Generate plots
    generate_plots()


def generate_plots():
    """Generate selected plots for current results."""
    global current_results, current_config
    
    if current_results is None:
        return
    
    widgets_dict['results_plots'].clear_output()
    
    with widgets_dict['results_plots']:
        import matplotlib.pyplot as plt
        
        selected_plots = list(widgets_dict['selected_plots'].value)
        dpi = widgets_dict['dpi'].value
        color_palette = widgets_dict['color_palette'].value
        
        print(f"Generating {len(selected_plots)} plots...\n")
        
        # Set style
        from dashboard.plots import set_plot_style
        set_plot_style(color_palette)
        
        for plot_name in selected_plots:
            try:
                if plot_name in EXTENDED_PLOT_REGISTRY:
                    plot_func = EXTENDED_PLOT_REGISTRY[plot_name]
                    
                    # Call appropriate plot function
                    if isinstance(current_results, dict):  # Multi-model
                        if plot_name in ['violin', 'box', 'radar_chart', 'model_size_vs_performance', 'pareto_front']:
                            plot_func(current_results, dpi=dpi)
                        else:
                            # Plot for first model
                            first_result = list(current_results.values())[0]
                            if plot_name in ['training_curves', 'learning_curve', 'convergence_analysis']:
                                plot_func(first_result.training_history, dpi=dpi)
                            else:
                                plot_func(first_result.evaluation, dpi=dpi)
                    
                    elif isinstance(current_results, list):  # Multi-seed
                        # Use first result
                        if plot_name in ['training_curves', 'learning_curve']:
                            plot_func(current_results[0].training_history, dpi=dpi)
                        else:
                            plot_func(current_results[0].evaluation, dpi=dpi)
                    
                    else:  # Single experiment
                        if plot_name in ['training_curves', 'learning_curve']:
                            plot_func(current_results.training_history, dpi=dpi)
                        else:
                            plot_func(current_results.evaluation, dpi=dpi)
                    
                    plt.show()
                    print(f"✅ {plot_name}")
                    
            except Exception as e:
                print(f"⚠️  Failed to generate {plot_name}: {str(e)}")
        
        print("\n✅ Plot generation complete!")


# Attach button callbacks
widgets_dict['button_run_experiment'].on_click(on_run_experiment_clicked)
widgets_dict['button_save_config'].on_click(on_save_config_clicked)
widgets_dict['button_load_config'].on_click(on_load_config_clicked)
widgets_dict['button_reset_defaults'].on_click(on_reset_defaults_clicked)

# Display buttons
buttons_box = widgets.HBox([
    widgets_dict['button_run_experiment'],
    widgets_dict['button_save_config'],
    widgets_dict['button_load_config'],
    widgets_dict['button_reset_defaults'],
], layout=widgets.Layout(justify_content='space-around', padding='20px'))

display(buttons_box)

# Display status output
display(widgets.HTML("<h3>Status Output</h3>"))
display(widgets_dict['status_output'])

## Cell 9: Real-Time Progress Display

Monitor experiment progress during execution.

In [ ]:
# Display progress widgets
display(widgets.HTML("<h3>Training Progress</h3>"))
display(widgets_dict['progress_bar'])
display(widgets_dict['live_metrics'])

## Cell 10: Results Dashboard

View experiment results, statistics, and visualizations.

In [ ]:
# Display results
display(widgets.HTML("<h3>Results Summary</h3>"))
display(widgets_dict['results_summary'])

display(widgets.HTML("<h3>Visualizations</h3>"))
display(widgets_dict['results_plots'])

## Cell 11: Export Results

Export results to various formats for publication and further analysis.

In [ ]:
# Export functions
import json
import pandas as pd
import torch

def export_results_to_csv(filename='results/experiment_results.csv'):
    """Export results to CSV."""
    global current_results
    
    if current_results is None:
        print("⚠️ No results to export. Run an experiment first.")
        return
    
    try:
        os.makedirs('results', exist_ok=True)
        
        if isinstance(current_results, dict):  # Multi-model
            data = []
            for model_name, result in current_results.items():
                row = {
                    'model': model_name,
                    'eta_top1': result.evaluation.eta_top1,
                    'accuracy_top1': result.evaluation.accuracy_top1,
                    'accuracy_top4': result.evaluation.accuracy_top4,
                    'eta_random_1': result.evaluation.eta_random_1,
                    'execution_time': result.execution_time,
                }
                data.append(row)
            
            df = pd.DataFrame(data)
            df.to_csv(filename, index=False)
            print(f"✅ Results exported to: {filename}")
            
        else:  # Single or multi-seed
            if isinstance(current_results, list):
                result = current_results[0]
            else:
                result = current_results
            
            data = {
                'metric': ['eta_top1', 'eta_top2', 'eta_top4', 'eta_top8',
                          'accuracy_top1', 'accuracy_top2', 'accuracy_top4', 'accuracy_top8'],
                'value': [
                    result.evaluation.eta_top1,
                    result.evaluation.eta_top2,
                    result.evaluation.eta_top4,
                    result.evaluation.eta_top8,
                    result.evaluation.accuracy_top1,
                    result.evaluation.accuracy_top2,
                    result.evaluation.accuracy_top4,
                    result.evaluation.accuracy_top8,
                ]
            }
            
            df = pd.DataFrame(data)
            df.to_csv(filename, index=False)
            print(f"✅ Results exported to: {filename}")
            
    except Exception as e:
        print(f"❌ Export failed: {str(e)}")


def export_results_to_json(filename='results/experiment_results.json'):
    """Export results to JSON."""
    global current_results
    
    if current_results is None:
        print("⚠️ No results to export. Run an experiment first.")
        return
    
    try:
        os.makedirs('results', exist_ok=True)
        
        if isinstance(current_results, dict):  # Multi-model
            export_data = {}
            for model_name, result in current_results.items():
                export_data[model_name] = result.to_dict()
        elif isinstance(current_results, list):  # Multi-seed
            export_data = {
                'results': [r.to_dict() for r in current_results],
                'aggregated': aggregate_results(current_results)
            }
        else:  # Single
            export_data = current_results.to_dict()
        
        with open(filename, 'w') as f:
            json.dump(export_data, f, indent=2, default=str)
        
        print(f"✅ Results exported to: {filename}")
        
    except Exception as e:
        print(f"❌ Export failed: {str(e)}")


def save_trained_model(filename='results/trained_model.pth'):
    """Save trained model."""
    global current_results
    
    if current_results is None:
        print("⚠️ No model to save. Run an experiment first.")
        return
    
    try:
        os.makedirs('results', exist_ok=True)
        
        if isinstance(current_results, dict):
            result = list(current_results.values())[0]
        elif isinstance(current_results, list):
            result = current_results[0]
        else:
            result = current_results
        
        if result.model_state:
            torch.save(result.model_state, filename)
            print(f"✅ Model saved to: {filename}")
        else:
            print("⚠️ No model state available.")
            
    except Exception as e:
        print(f"❌ Save failed: {str(e)}")


def generate_latex_table():
    """Generate LaTeX table for publications."""
    global current_results
    
    if current_results is None:
        print("⚠️ No results to export. Run an experiment first.")
        return
    
    try:
        print("\\begin{table}[htbp]")
        print("\\centering")
        print("\\caption{Experiment Results}")
        print("\\label{tab:results}")
        
        if isinstance(current_results, dict):  # Multi-model
            print("\\begin{tabular}{lccc}")
            print("\\toprule")
            print("Model & $\\eta_{\\text{top-1}}$ & Accuracy & Time (s) \\\\")
            print("\\midrule")
            
            for model_name, result in current_results.items():
                print(f"{model_name} & {result.evaluation.eta_top1:.4f} & "
                      f"{result.evaluation.accuracy_top1:.3f} & {result.execution_time:.1f} \\\\")
            
            print("\\bottomrule")
            print("\\end{tabular}")
        else:
            if isinstance(current_results, list):
                result = current_results[0]
            else:
                result = current_results
            
            print("\\begin{tabular}{lc}")
            print("\\toprule")
            print("Metric & Value \\\\")
            print("\\midrule")
            print(f"$\\eta_{{\\text{{top-1}}}}$ & {result.evaluation.eta_top1:.4f} \\\\")
            print(f"$\\eta_{{\\text{{top-4}}}}$ & {result.evaluation.eta_top4:.4f} \\\\")
            print(f"Accuracy (Top-1) & {result.evaluation.accuracy_top1:.3f} \\\\")
            print(f"Accuracy (Top-4) & {result.evaluation.accuracy_top4:.3f} \\\\")
            print("\\bottomrule")
            print("\\end{tabular}")
        
        print("\\end{table}")
        print("\n✅ LaTeX table generated!")
        
    except Exception as e:
        print(f"❌ Generation failed: {str(e)}")


# Create export buttons
btn_export_csv = widgets.Button(description="Export to CSV", button_style='info')
btn_export_json = widgets.Button(description="Export to JSON", button_style='info')
btn_save_model = widgets.Button(description="Save Model", button_style='success')
btn_latex_table = widgets.Button(description="Generate LaTeX", button_style='warning')

btn_export_csv.on_click(lambda b: export_results_to_csv())
btn_export_json.on_click(lambda b: export_results_to_json())
btn_save_model.on_click(lambda b: save_trained_model())
btn_latex_table.on_click(lambda b: generate_latex_table())

export_buttons = widgets.HBox([btn_export_csv, btn_export_json, btn_save_model, btn_latex_table],
                              layout=widgets.Layout(justify_content='space-around', padding='20px'))

display(widgets.HTML("<h3>Export Options</h3>"))
display(export_buttons)

---

## 📚 Learning Guide

### Probe Types Explained

1. **Continuous**: Random phases in [0, 2π). Best for theoretical studies.
2. **Binary**: Phases {0, π}. Simplest hardware implementation.
3. **2-bit**: Phases {0, π/2, π, 3π/2}. Good balance of performance and simplicity.
4. **Hadamard**: Structured orthogonal patterns. Excellent diversity.
5. **Sobol**: Low-discrepancy quasi-random. Better coverage than random.
6. **Halton**: Another quasi-random sequence. Similar to Sobol.

### Model Architecture Guidelines

- **Wider networks** (e.g., DoubleWide): More capacity, risk of overfitting
- **Deeper networks** (e.g., VeryDeep): Better feature extraction, harder to train
- **Pyramidal** (e.g., Pyramid): Natural information compression
- **Hourglass**: Information bottleneck for robust features
- **ResNet-style**: Same width, easier gradient flow

### Key Parameter Interactions

- **M/K ratio**: Critical for performance. Lower ratio = harder problem.
- **Learning rate**: Most important hyperparameter. Start with 1e-3.
- **Dropout**: Use 0.1-0.2 for regularization. Higher values for larger models.
- **Batch size**: Larger = more stable gradients. Smaller = better generalization.

### Typical Workflows

1. **Quick Test**: Default settings, 1 epoch, check if system works
2. **Architecture Search**: Compare multiple models, same data/training
3. **Hyperparameter Tuning**: Fix architecture, sweep learning rates
4. **Statistical Validation**: Multi-seed runs for confidence intervals
5. **Publication Results**: Best config, full training, all plots

---

## 🛠️ Troubleshooting

**Out of memory?**
- Reduce batch_size
- Use smaller model
- Reduce n_train

**Training too slow?**
- Reduce n_epochs
- Use smaller dataset
- Use simpler model

**Poor performance?**
- Increase model capacity
- Try different probe types
- Adjust learning rate
- More training data

**Overfitting?**
- Increase dropout
- Add weight_decay
- Reduce model size
- More training data

---

## 📖 References

For more information, see:
- `dashboard/README.md` - Detailed documentation
- `EXTENSION_GUIDE.md` - How to extend the system
- `USAGE_EXAMPLES.md` - Usage examples

---

**Happy Researching! 🚀**